In [ ]:
from standard_functions import *
path = os.getcwd()
data_path = os.path.join(os.path.dirname(path), 'Data') 

BloombergData = pd.read_csv(data_path + "/BloombergData_Swap_Features.csv")
preData = pd.read_csv(data_path + "/TestData_Swap_Features_pre.csv")
postData = pd.read_csv(data_path + "/TestData_Swap_Features_post.csv")

insample = np.array(BloombergData.iloc[:,2:].reset_index(drop=True)) 
insample_scaled = [x/100 for x in insample]
insample_tensor = torch.from_numpy(np.float32(insample_scaled))

X_train1, X_val1 = train_validation_split(BloombergData.reset_index(drop=True), 0.1) 
X_train2  = np.array(X_train1.iloc[:,2:].reset_index(drop=True)) 
X_train_scaled = [x/100 for x in X_train2]
X_train_tensor = torch.from_numpy(np.float32(X_train_scaled))

X_val2  = np.array(X_val1.iloc[:,2:].reset_index(drop=True)) 
X_val_scaled = [x/100 for x in X_val2]
X_val_tensor = torch.from_numpy(np.float32(X_val_scaled))

preX_test2  = np.array(preData.iloc[:,2:].reset_index(drop=True)) 
preX_test_scaled = [x/100 for x in preX_test2]
preX_test_tensor = torch.from_numpy(np.float32(preX_test_scaled))

postX_test2  = np.array(postData.iloc[:,2:].reset_index(drop=True)) 
postX_test_scaled = [x/100 for x in postX_test2]
postX_test_tensor = torch.from_numpy(np.float32(postX_test_scaled))


In [ ]:
class CustomLoss(nn.Module):
    def __init__(self): 
        super().__init__()
    
    def forward(self, yhat, y, L, w):  
        mse = torch.mean(torch.sum(torch.pow(yhat - y, 2),1)/8)
        arbitrage_loss = torch.mean(torch.sum(torch.pow(L, 2), 1)/30)
       
        return mse + (w*arbitrage_loss)

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(8, 3, bias = False)
        )
        
        self.sigma = nn.Sequential(
            nn.Linear(3, 6, bias = False),
            centered_softmax(),                 
            nn.Linear(6, 6, bias = False)
        )

        self.mu = nn.Sequential(
            nn.Linear(3, 3, bias = False),
            centered_softmax(),
            nn.Linear(3, 3, bias = False)
        )

        self.decoder = nn.Sequential(
            nn.Linear(4, 10, bias = False),
            centered_softmax(),
            nn.Linear(10, 1, bias = False)
        )
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)

    def forward(self, x):
        
        encoded = self.encoder(x) 
        
        mat = torch.arange(1, 31, dtype=torch.float32).repeat(len(encoded))
        
        # y for 1-30
        x1 = encoded.repeat_interleave(30, dim=0)
        encoded_mat = torch.cat([x1, mat.unsqueeze(1)], dim=1)     
        y = self.decoder(encoded_mat).reshape(len(encoded), 30)

        # r i.e y[:,0]
        mat0 = torch.zeros(len(encoded),1)
        encoded_mat_0 = torch.cat([encoded, mat0], dim=1) 
        r = self.decoder(encoded_mat_0)
        
        sigma_1, sigma_2, sigma_3, rho12, rho13, rho23 = torch.split(self.sigma(encoded), 1, dim=1)
        # K network for mu
        mu = self.mu(encoded)

        #Remove explosion in the first 1-5 epochs. Tau 30 + bad weight ini cause P > 1000. Training gets stuck. 
        p = torch.exp(torch.clamp((-mat.reshape(y.shape[0],30) * y), min = -80, max = 80))
        
        p_cumsum = torch.cumsum(p, dim=1)  

        s = (1 - p) / p_cumsum
        
        return r, y, p, sigma_1, sigma_2, sigma_3, rho12, rho13, rho23, mu, encoded, encoded_mat, s, mat    

In [ ]:
def arb_loss(r, y, p, sigma_1, sigma_2, sigma_3, rho12, rho13, rho23, mu, encoded_mat, tau, model):
    N = len(y)
    
    r = r.reshape(N, 1)                  
    r_long = r.repeat_interleave(30, dim=0) 
    y_long = y.reshape(N*30, 1)
    tau_long =tau.reshape(N*30, 1) 
    mu_long = mu.repeat_interleave(30, dim=0)     
    
    grad_zy, dy_dm   = grad_latent_3factor(encoded_mat, model) 
    
    grad_zmu = torch.matmul(grad_zy.unsqueeze(1), mu_long.unsqueeze(2)).squeeze(-1)
     
    hess_z  = hess_latent_3factor(encoded_mat, model) 
    sigma_long = build_sigma_matrix_3factor(sigma_1, sigma_2, sigma_3, rho12, rho13, rho23)
    sigma_hess_sigma = torch.matmul(sigma_long.transpose(1, 2), torch.matmul(hess_z, sigma_long)) 
    trace_hess = 0.5 * torch.einsum('bii->b', sigma_hess_sigma).unsqueeze(-1)
    
    grad_grad = torch.matmul(grad_zy.unsqueeze(-1), grad_zy.unsqueeze(-1).transpose(1, 2))
    sigma_grad_grad_sigma = torch.matmul(sigma_long.transpose(1, 2), torch.matmul(grad_grad, sigma_long)) 
    trace_grad_grad = 0.5 * torch.einsum('bii->b', sigma_grad_grad_sigma).unsqueeze(-1)
    
    final_term =  -r_long + y_long + tau_long*(dy_dm - grad_zmu - trace_hess) + (tau_long**2)*trace_grad_grad   
        
    return final_term.reshape(len(y), 30)*p

In [ ]:
class CustomRMSE(nn.Module):
    def __init__(self): 
        super().__init__()
    
    def forward(self, yhat, y):  
        mse = torch.mean(torch.sum(torch.pow(yhat - y, 2),1)/8)

        return mse 

In [ ]:
# torch.manual_seed(3)

# model = Autoencoder()   
# criterion= CustomLoss()
# rmse = CustomRMSE()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# scheduler = LR_Scheduler(optimizer, percentage=0.9, interval = 50)

# # Training
# num_epochs = 4993 
# batch_size = 32
# data_loader = torch.utils.data.DataLoader(X_train_tensor, batch_size=batch_size, shuffle=True)

# swap_mats = [1, 2, 3, 5, 10, 15, 20, 30]
# swap_mats0 = [i-1 for i in swap_mats]

# arb_losses_list = []
# losses_list = []
# val_loss_list = []
# losses_list_rmse = []
# val_loss_list_rmse = []

# for epoch in range(num_epochs):
#     arb_loss_epoch = 0
#     loss_epoch = 0

#     for batch in data_loader:
        
#         # Forward pass
#         r, y, p, sigma_1, sigma_2, sigma_3, rho12, rho13, rho23, mu, encoded, encoded_mat, reconstructed, mat = model(batch)

#         arbitrage_loss = arb_loss(r, y, p, sigma_1, sigma_2, sigma_3, rho12, rho13, rho23, mu, encoded_mat, mat, model)
                                      
#         s_final = reconstructed[:, swap_mats0]
        
#         loss = criterion(s_final, batch, arbitrage_loss, 1)
        
#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
        
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
#         optimizer.step()
        
#         arb_loss_epoch += torch.sum(arbitrage_loss)
#         loss_epoch += loss
    
#     r, y, p, sigma_1, sigma_2, sigma_3, rho12, rho13, rho23, mu, encoded, encoded_mat, reconstructed, mat = model(X_val_tensor)
#     s_final = reconstructed[:, swap_mats0]
#     arbitrage_loss = arb_loss(r, y, p, sigma_1, sigma_2, sigma_3, rho12, rho13, rho23, mu, encoded_mat, mat, model)
#     loss_val = criterion(s_final, X_val_tensor, arbitrage_loss, 1)
#     loss_val_rmse = rmse(s_final, X_val_tensor)

#     r, y, p, sigma_1, sigma_2, sigma_3, rho12, rho13, rho23, mu, encoded, encoded_mat, reconstructed, mat = model(X_train_tensor)
#     s_final = reconstructed[:, swap_mats0]
#     arbitrage_loss = arb_loss(r, y, p, sigma_1, sigma_2, sigma_3, rho12, rho13, rho23, mu, encoded_mat, mat, model)
#     loss_train = criterion(s_final, X_train_tensor, arbitrage_loss, 1)
#     loss_train_rmse = rmse(s_final, X_train_tensor)

#     losses_list.append(loss_train.item())
#     losses_list_rmse.append(loss_train_rmse.item())
#     val_loss_list.append(loss_val.item())
#     val_loss_list_rmse.append(loss_val_rmse.item())   
#     #arb_losses_list.append(arb_loss_epoch.item())
    
#     scheduler.step()
#     current_lr = optimizer.param_groups[0]["lr"] 
    
#     if (epoch + 1) % 100 == 0:
#         print(f"Epoch [{epoch + 1}/{num_epochs}], LR:{current_lr:.8f}, Loss: {loss_train.item():.8f}, Arb:{arb_loss_epoch:.8f} ")

In [ ]:
name = "3factor_minArb"
# torch.save(model.state_dict(), f"models/{name}.pth")
# print(f"Model saved successfully as models/{name}.pth")
model2 = Autoencoder()       # Ensure Autoencoder class is defined
model2.load_state_dict(torch.load(f"models/{name}.pth"))